In [13]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


In [27]:
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id":'productTitle'})       
        title_value = title.text
        title_string = title_value.strip()
    except AttributeError:
        title_string = ""
    return title_string

def get_price(soup):
    try:
        price_tag = soup.find("span", attrs={'class':'a-offscreen'})
        if price_tag:
            price = price_tag.text.strip()
            cleaned_price = ''.join(filter(str.isdigit, price))
            try:
                price_in_dollars = float(cleaned_price) / 100  
                price_in_rupees = price_in_dollars * 83.46
                return f"{price_in_rupees:.2f} INR"
            except ValueError:
                return ""
    except AttributeError:
        pass

    try:
        price_tag = soup.find("span", attrs={'id':'a-span12'})
        if price_tag:
            price = price_tag.text.strip()
            cleaned_price = ''.join(filter(str.isdigit, price))
            try:
                price_in_dollars = float(cleaned_price) / 100  
                price_in_rupees = price_in_dollars * 83.46
                return f"{price_in_rupees:.2f} INR"
            except ValueError:
                return ""
    except AttributeError:
        pass
    return ""


def get_rating(soup):
    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()   
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	
    return rating

def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()
    except AttributeError:
        review_count = ""	
    return review_count

def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available
    
if __name__ == '__main__':

    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    URL = "https://www.amazon.com/s?k=Indian+Flag&crid=2Z4O4OM17BU8S&sprefix=indian+flag%2Caps%2C305&ref=nb_sb_noss_1"

    webpage = requests.get(URL, headers=HEADERS)

    soup = BeautifulSoup(webpage.content, "html.parser")

    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    links_list = []

    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
   
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

ConnectionError: HTTPSConnectionPool(host='www.amazon.comhttps', port=443): Max retries exceeded with url: //aax-us-iad.amazon.com/x/c/RMCTKbgmpxmJSQXgZv5aAPIAAAGP2VFIEgEAAAH2AQBvbm9fdHhuX2JpZDQgICBvbm9fdHhuX2ltcDEgICCUTA_4/https://www.amazon.com/Tear-Resistant-Resistant-Polyester-Embroidered-Grommerts/dp/B0CN2QTWW6/ref=sxin_25_sbv_search_btf?content-id=amzn1.sym.5cde1a09-4942-4242-87c5-e66d2d3b6a3c%3Aamzn1.sym.5cde1a09-4942-4242-87c5-e66d2d3b6a3c&crid=2Z4O4OM17BU8S&cv_ct_cx=Indian+Flag&dib=eyJ2IjoiMSJ9.O64T59D1e1L3dpWOqH_pmg.TOOgimpbLqrvGVJtE_dbaD7yrowiKYuT3AxOiMgmrew&dib_tag=se&keywords=Indian+Flag&pd_rd_i=B0CN2QTWW6&pd_rd_r=505766fb-3e38-4266-83cf-edc50ea99389&pd_rd_w=BgRkC&pd_rd_wg=j4tS8&pf_rd_p=5cde1a09-4942-4242-87c5-e66d2d3b6a3c&pf_rd_r=VDZVF0PQ08DVF23CBW1G&qid=1717337933&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sprefix=indian+flag%2Caps%2C305&sr=1-1-5190daf0-67e3-427c-bea6-c72c1df98776 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001FA75975C5

In [23]:
amazon_df

,title,price,rating,reviews,availability
0,Double Sided India Flag 3x5ft Heavy Duty Delux...,1668.37 INR,4.5 out of 5 stars,103 ratings,
1,G128 India Indian Flag | 3x5 Ft | LiteWeave Pr...,1084.15 INR,4.5 out of 5 stars,105 ratings,In Stock
2,G128 Indiana State Flag | 4x6 Ft | ToughWeave ...,2332.71 INR,4.6 out of 5 stars,270 ratings,In Stock
3,"Premium USA Indian Flag 3X5Ft Large New, Both ...",833.77 INR,4.7 out of 5 stars,210 ratings,Only 12 left in stock - order soon.
4,Anley Fly Breeze 3x5 Foot India Flag - Vivid C...,580.05 INR,4.7 out of 5 stars,"1,383 ratings",In Stock
...,...,...,...,...,...
59,Indian Banner Flag 2x8ft Garage Decor Wall Man...,1501.45 INR,5.0 out of 5 stars,1 rating,In Stock
60,"Premium USA Indian Flag 3X5Ft Large New, Both ...",833.77 INR,4.7 out of 5 stars,210 ratings,Only 12 left in stock - order soon.
61,"Native American Indian Healing flag 3x5FT, Hea...",917.23 INR,4.8 out of 5 stars,6 ratings,Only 2 left in stock - order soon.
62,AZ FLAG - USA Indian Chief Flag - 3x5 Ft - 100...,1080.81 INR,4.3 out of 5 stars,110 ratings,Only 8 left in stock - order soon.
